#### Simple Validators

A quick way to add a __"validation" routine__ to an attribute is to use the `validates()` decorator. An _attribute validator_ can raise an exception, halting the process of _mutating the attribute's value_, or can _change the given value into something different_. _Validators_, like all attribute extensions, are __only called by normal userland code__; they are __not issued when the ORM is populating the object__.

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import declarative_base, validates, relationship

In [2]:
Base = declarative_base()

In [3]:
class EmailAddress(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email = Column(String)
    
    @validates("email")
    def validate_email(self, key, address):
        if "@" not in address.email:
            raise ValueError("failed simplified email validation")
        return address

_Validators_ also `receive` _collection append events_, when _items_ are __added to a collection__.

In [4]:
class User(Base):
    __tablename__ = "user"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address")
    
    @validates("addresses")
    def validate_addresses(self, key, address):
        if "@" not in address.email:
            raise ValueError("failed simplified email validation")
        return address

The _validation function_ __by default does not get emitted__ for __collection remove events__, as the typical expectation is that a _value being discarded doesn't require validation_. However, `validates()` supports reception of these events by specifying `include_removes=True` to the decorator. When this _flag is set_, the _validation function_ __must receive an additional boolean argument__ which if True indicates that the operation is a removal.

In [5]:
class UserRedundant(Base):
    __tablename__ = "user_redundant"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address")
    
    @validates("addresses", include_removes=True)
    def validate_addresses(self, key, address):
        if "@" not in address.email:
            raise ValueError("failed simplified email validation")
        return address

The case where _mutually dependent validators_ are _linked via a backref_ can also be tailored, using the `include_backrefs=False` option; this option, when set to `False`, __prevents a validation function from emitting__ if the event occurs as a result of a backref.

In [6]:
class Address(Base):
    __tablename__ = "address_backref"
    
    id = Column(Integer, primary_key=True)
    email = Column(String)
    user_id = Column(Integer, ForeignKey("user_backref.id"), nullable=False)
    
    @validates("email")
    def validate_email(self, key, address):
        if "@" not in address.email:
            raise ValueError("failed simplified email validation")
        return address

In [7]:
class UserBackref(Base):
    __tablename__ = "user_backref"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", backref="user")
    
    @validates("addresses", include_backrefs=False)
    def validate_addresses(self, key, address):
        if "@" not in address.email:
            raise ValueError("failed simplified email validation")
        return address

Above, if we were to assign to `Address.user` as in `some_address.user = some_user`, the `validate_address()` function __would not be emitted__, even though an append occurs to `some_user.addresses` - the event is _caused by a backref_.

Note that the `validates()` decorator is a _convenience function_ built on top of _attribute events_. An application that requires _more control over configuration of attribute change behavior_ can make use of this system, described at `AttributeEvents`.